In [1]:
%%time
import pandas as pd
import numpy as np
import gensim
from pipeline_utilities_insincere_questions import *
from sklearn.model_selection import train_test_split
from get_good_and_bad_words import get_good_and_bad_words
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model.init_sims(replace=True)

[nltk_data] Downloading package punkt to /Users/lduque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
CPU times: user 7min 29s, sys: 10.5 s, total: 7min 40s
Wall time: 7min 41s


In [2]:
data = pd.read_csv('train_inisincere.csv', index_col='qid')
X_test = pd.read_csv('test_insincere.csv', index_col='qid')
X = data[['question_text']]
y = data.target
X_train, X_dictionary, y_train, y_dictionary = train_test_split(X, y, test_size=0.2, random_state=42)
data_dictionary =  pd.concat([X_dictionary, y_dictionary], axis=1)

In [3]:
X_dictionary.shape

(261225, 1)

In [4]:
X_train.shape

(1044897, 1)

In [5]:
X_test.shape

(56370, 1)

In [6]:
%%time
good_words, bad_words = get_good_and_bad_words(data_dictionary)
bad_words = list(bad_words.sort_values(ascending=False).index)
good_words = list(good_words.sort_values(ascending=False).index)

In [7]:
size_sublists = 300
chunk_bad_words = [bad_words[x:x+size_sublists] for x in xrange(0, len(bad_words), size_sublists)]
chunk_good_words = [good_words[x:x+size_sublists] for x in xrange(0, len(good_words), size_sublists)]
worst_sentence = ' '.join(chunk_bad_words[0])
best_sentence = ' '.join(chunk_good_words[0])

In [8]:
print worst_sentence

muslims liberals trump white women castrated indians men americans democrats christians hillary people jews whites shithole blacks hindus black muslim racist obama brainwashed castrate feminists atheists girls gay realize castrating fuck hate bhakts stupid conservatives enslaved leftists asians republicans supporters shabbat clinton rape liberal gujaratis sex muhajirs donald xenophobic pakistanis fucking modi gays incest indian clintons palestinians islam chinese raping rapists superior mullahs hypocrites quora crimes israel raped shit racists homosexuals brahmins president democrat assholes american jewish dumb ignorant filth america kill holocaust racism asian homosexual islamization ass hindu dick idiots terrorists zealanders israelis homosexuality males deny europeans husbands lynched believe tamilians minorities guys wives illegals violent illiterate islamist tamils dont sister troll ugly kannadigas guns boys zionism fucked bullshit pakistani transgender losers easterners stereoty

In [9]:
print best_sentence

best good get life someone engineering difference use business job online college book company work used books exam computer possible study different student career university learn process rank experience prepare app ways design one software admission find marks data degree learning important tips write course way marketing buy water apply year website jee iit web mobile engineer required affect long cost major science start program technology would field examples management physics google card android system institute interview account movie number mean score phone improve time salary main writing available product month exams favorite function test youtube studying coaching programming invest download earn market top current energy colleges laptop causes meaning apps application happen bank pursue visa investment first types without school cse characteristics series preparation distance advice options international students class research air skills iphone role amazon machine car tr

In [10]:
X=[]
y=[]
data=[]
good_words=[]
bad_words=[]
X_dictionary =[]
y_dictionary =[]

In [11]:
%%time
m = stringTransformer(colname='question_text', model=model)
encoded_strings_train = m.transform(X_train)
encoded_strings_test = m.transform(X_test)

CPU times: user 12min 19s, sys: 8.14 s, total: 12min 28s
Wall time: 12min 30s


In [ ]:
%%time
m = stringComparison(col='question_text', sentence=worst_sentence, suffix='_worst', model=model, norm_model=norm_model)
comparison_with_worst_sentence_train = m.transform(X_train)
comparison_with_worst_sentence_test = m.transform(X_test)

In [ ]:
%%time
m = stringComparison(col='question_text', sentence=best_sentence, suffix='_best',model=model, norm_model=norm_model)
comparison_with_best_sentence_train = m.transform(X_train)
comparison_with_best_sentence_test = m.transform(X_test)

In [ ]:
train_features = pd.concat([X_train.question_text, encoded_strings_train, comparison_with_worst_sentence_train, comparison_with_best_sentence_train], axis=1)
test_features = pd.concat([X_test.question_text, encoded_strings_test, comparison_with_worst_sentence_test, comparison_with_best_sentence_test], axis=1)

In [ ]:
train_features.to_csv("train_features.csv")
test_features.to_csv("test_features.csv")